In [1]:
import nemoguardrails as ng

In [2]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model_id = "../models/TheBloke/Mistral-7B-Instruct-v0.2-GPTQ/gptq-4bit-32g-actorder_True/"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


In [4]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, device=0)

In [5]:
hf = HuggingFacePipeline(pipeline=pipe)

In [27]:
from nemoguardrails.llm.helpers import get_llm_instance_wrapper
from nemoguardrails.llm.providers import register_llm_provider

HFPipeline = get_llm_instance_wrapper(
    llm_instance=hf, llm_type="gptq-4bit-32g-actorder_True"
)

register_llm_provider("gptq-4bit-32g-actorder_True", HFPipeline)

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
template = """<s>[INST] {question} [/INST]"""
prompt = PromptTemplate.from_template(template)

In [8]:
chain = prompt | hf

In [9]:
question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Electroencephalography (EEG) is a diagnostic method used to evaluate the electrical activity of the brain. It involves placing electrodes on the scalp to measure the voltage fluctuations produced by ionic currents within the neurons of the brain. These electrical signals can provide information about the brain's electrical activity, which can help diagnose various neurological conditions, such as epilepsy, sleep disorders, and brain injuries. EEG is a non-invasive, painless procedure that can provide valuable insights into the functioning of the brain and help guide clinical decision-making.


In [28]:
colang_content = """
# define niceties
define user express greeting
    "hello"
    "hi"
    "what's up?"

define flow greeting
    user express greeting
    bot express greeting
    bot ask how are you

# define limits
define user ask sports
    "what is your favorite football team?"
    "who is Rafael Nadal?"
    "football player"
    "tennis player"
    "match result"
    
define bot answer sports
    "I'm a shopping assistant, I don't like to talk of sports."

define flow sports
    user ask sports
    bot answer sports
    bot offer help
"""

yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
# - type: embeddings
#   engine: openai
#   model: text-embedding-ada-002
"""

In [29]:
# initialize rails config
config = ng.RailsConfig.from_content(
    colang_content=colang_content
)

In [30]:
llm = ng.LLMRails(llm=HFPipeline, config=config)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
response = llm.generate_async(prompt="Hello, how are you?")

In [32]:
print(response)
# Output: Hello, I'm fine, thank you. Are you a fan of any sports?

<coroutine object LLMRails.generate_async at 0x7f7c752d1f80>


In [33]:
response

<coroutine object LLMRails.generate_async at 0x7f7c752d1f80>